# Constant Maturity Contracts

A special type of continuous contract is a constant maturity contract
as discussed at the end of the continuous contract notebook.
This notebook demonstrates how that works in practice.

Suppose the goal of the analysis is to understand trends and abnormalities at the front of the yield curve
to create a trading strategy. The first step is to prepare price data from the `SR3` contract. Rather
than jumping straight to constant maturity prices, let's first look at continuous prices by expiration
to see why that simpler approach may not be sufficient for our purposes.

The following uses the ideas in Steve Moffitt's book The Strategic Analysis Of Financial Markets, specifically Chapters 13 and 14 in Volume 2.


In [1]:
import datetime
import sys
from itertools import cycle
from zoneinfo import ZoneInfo

import databento as db
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.decomposition import FastICA

from finm37000 import (
    constant_maturity_splice,
    get_databento_api_key,
    get_official_stats,
    get_roll_spec,
    temp_env,
    us_business_day,
)

px.defaults.color_discrete_sequence = px.colors.qualitative.Set3
color_palette = cycle(px.defaults.color_discrete_sequence)

tz_chicago = ZoneInfo("America/Chicago")

sys.executable

'/Users/ericpatterson/.venv/finm37000-2025-09-17/bin/python'

In [2]:
with temp_env(DATABENTO_API_KEY=get_databento_api_key()):
    client = db.Historical()
start_of_this_year = (datetime.date(2025, 1, 1) + us_business_day).date()
yesterday = (datetime.datetime.now(tz=tz_chicago) - 2 * us_business_day).date()
cme = "GLBX.MDP3"

## Continous Prices By Expiration



In [3]:
product = "SR3"
roll_type = "c"
num_contracts = 10
continuous_at_expiration = tuple(
    f"{product}.{roll_type}.{i}" for i in range(num_contracts)
)
continuous_at_expiration

('SR3.c.0',
 'SR3.c.1',
 'SR3.c.2',
 'SR3.c.3',
 'SR3.c.4',
 'SR3.c.5',
 'SR3.c.6',
 'SR3.c.7',
 'SR3.c.8',
 'SR3.c.9')

In [4]:
ohlcv_continuous = client.timeseries.get_range(
    dataset=cme,
    schema="ohlcv-1d",
    symbols=continuous_at_expiration,
    stype_in="continuous",
    start=start_of_this_year,
    end=yesterday,
).to_df()

/var/folders/8y/2f_yzjcd2tz12vswl92n4vpw0000gp/T/ipykernel_18664/3892388200.py:1: BentoWarning: The streaming request contained one or more days which have reduced quality: 2025-09-17 (degraded), 2025-09-24 (degraded). See: https://databento.com/docs/api-reference-historical/metadata/metadata-get-dataset-condition
  ohlcv_continuous = client.timeseries.get_range(


In [5]:
plot_df = ohlcv_continuous.reset_index()
fig = px.line(
    plot_df,
    x="ts_event",
    y="close",
    color="symbol",
)
fig.update_layout(title_text=f"Unadjusted spliced futures, {product}")
fig.show()

### Poll

What issues do you notice in this graph that could affect using it to analyze the front of the yield curve?


### Detour on back month prices


Why does some of the back months look so linear for extended times? We won't correct this because we are focusing on the constant maturity method, but
let's take a brief detour to interrogate the data. It reveals important issues that emerge when managing futures data.

First, let's see what the data around June/July looks like for `SR3.c.9` when there is a long line that does not track the other continuous contract prices.

In [6]:
potential_problem = "SR3.c.9"
ohlcv_continuous[ohlcv_continuous["symbol"] == potential_problem]

,rtype,publisher_id,instrument_id,open,high,low,close,volume,symbol
ts_event,,,,,,,,,
2025-01-15 00:00:00+00:00,35,1,254273,95.885,96.000,95.885,95.975,259247,SR3.c.9
2025-01-16 00:00:00+00:00,35,1,254273,95.970,96.015,95.930,95.995,234875,SR3.c.9
2025-01-17 00:00:00+00:00,35,1,254273,95.995,96.015,95.955,95.955,149210,SR3.c.9
2025-01-19 00:00:00+00:00,35,1,254273,95.955,95.955,95.945,95.955,998,SR3.c.9
2025-01-20 00:00:00+00:00,35,1,254273,95.955,96.010,95.955,96.010,43530,SR3.c.9
...,...,...,...,...,...,...,...,...,...
2025-10-28 00:00:00+00:00,35,1,254277,96.800,96.805,96.775,96.795,164407,SR3.c.9
2025-10-29 00:00:00+00:00,35,1,254277,96.795,96.795,96.650,96.670,287271,SR3.c.9
2025-10-30 00:00:00+00:00,35,1,254277,96.670,96.690,96.625,96.650,292128,SR3.c.9


There seem to be missing days. What does the roll spec say about what actual contract should be used there?

In [7]:
roll_spec = client.symbology.resolve(
    dataset=cme,
    symbols=potential_problem,
    stype_in="continuous",
    stype_out="instrument_id",
    start_date=start_of_this_year,
    end_date=yesterday,
)
roll_spec["result"][potential_problem]

[{'d0': '2025-01-02', 'd1': '2025-01-15', 's': '42054065'},
 {'d0': '2025-01-15', 'd1': '2025-01-31', 's': '254273'},
 {'d0': '2025-01-31', 'd1': '2025-02-19', 's': '42155108'},
 {'d0': '2025-02-19', 'd1': '2025-03-19', 's': '254273'},
 {'d0': '2025-03-19', 'd1': '2025-03-31', 's': '254274'},
 {'d0': '2025-03-31', 'd1': '2025-04-16', 's': '42014666'},
 {'d0': '2025-04-16', 'd1': '2025-05-05', 's': '254274'},
 {'d0': '2025-05-05', 'd1': '2025-05-21', 's': '42000012'},
 {'d0': '2025-05-21', 'd1': '2025-06-18', 's': '254274'},
 {'d0': '2025-06-18', 'd1': '2025-06-30', 's': '254276'},
 {'d0': '2025-06-30', 'd1': '2025-07-16', 's': '42000173'},
 {'d0': '2025-07-16', 'd1': '2025-08-04', 's': '254276'},
 {'d0': '2025-08-04', 'd1': '2025-08-20', 's': '42032085'},
 {'d0': '2025-08-20', 'd1': '2025-09-17', 's': '254276'},
 {'d0': '2025-09-17', 'd1': '2025-09-29', 's': '254277'},
 {'d0': '2025-09-29', 'd1': '2025-10-15', 's': '42000061'},
 {'d0': '2025-10-15', 'd1': '2025-11-03', 's': '254277'},


In [8]:
potential_problem_id = 42000173
missing_ohlcv = client.timeseries.get_range(
    dataset=cme,
    schema="ohlcv-1d",
    symbols=potential_problem_id,
    stype_in="instrument_id",
    start=start_of_this_year,
    end=yesterday,
).to_df()
missing_ohlcv

/var/folders/8y/2f_yzjcd2tz12vswl92n4vpw0000gp/T/ipykernel_18664/3052674077.py:2: BentoWarning:

The streaming request contained one or more days which have reduced quality: 2025-09-17 (degraded), 2025-09-24 (degraded). See: https://databento.com/docs/api-reference-historical/metadata/metadata-get-dataset-condition



,rtype,publisher_id,instrument_id,open,high,low,close,volume,symbol
ts_event,,,,,,,,,
2025-07-24 00:00:00+00:00,35,1,42000173,96.170,96.170,96.170,96.170,1,42000173
2025-08-08 00:00:00+00:00,35,1,42000173,96.350,96.350,96.350,96.350,50,42000173
2025-08-12 00:00:00+00:00,35,1,42000173,96.365,96.370,96.350,96.370,102,42000173
2025-08-14 00:00:00+00:00,35,1,42000173,96.405,96.405,96.405,96.405,16,42000173
2025-08-19 00:00:00+00:00,35,1,42000173,96.275,96.285,96.275,96.285,200,42000173
2025-08-21 00:00:00+00:00,35,1,42000173,96.245,96.245,96.245,96.245,2,42000173
2025-08-22 00:00:00+00:00,35,1,42000173,96.335,96.345,96.335,96.345,6,42000173
2025-08-26 00:00:00+00:00,35,1,42000173,96.325,96.335,96.320,96.325,258,42000173
2025-08-27 00:00:00+00:00,35,1,42000173,96.330,96.330,96.330,96.330,1,42000173


This contract has no OHLC data during the period in question. How about other information? What does the instrument id correspond to? Does it have settlement
information?


In [9]:
raw_stats = client.timeseries.get_range(
    dataset=cme,
    schema="statistics",
    symbols=potential_problem_id,
    stype_in="instrument_id",
    start=start_of_this_year,
    end=yesterday,
)

/var/folders/8y/2f_yzjcd2tz12vswl92n4vpw0000gp/T/ipykernel_18664/37503047.py:1: BentoWarning:

The streaming request contained one or more days which have reduced quality: 2025-09-17 (degraded), 2025-09-24 (degraded). See: https://databento.com/docs/api-reference-historical/metadata/metadata-get-dataset-condition



In [10]:
old_defs = client.timeseries.get_range(
    dataset=cme,
    schema="definition",
    symbols=potential_problem_id,
    stype_in="instrument_id",
    start=start_of_this_year,
)
instrument_defs = client.timeseries.get_range(
    dataset=cme,
    schema="definition",
    symbols=potential_problem_id,
    stype_in="instrument_id",
    start=yesterday,
)
stats = get_official_stats(raw_stats.to_df(), instrument_defs.to_df())

In [11]:
stats

,,Settlement price,Cleared volume,Open interest,expiration
Trade date,Symbol,,,,
2025-01-02,SR3F6,32.500,NaN,NaN,2026-04-14 21:00:00+00:00
2025-01-03,SR3F6,35.750,NaN,NaN,2026-04-14 21:00:00+00:00
2025-01-06,SR3F6,31.500,NaN,NaN,2026-04-14 21:00:00+00:00
2025-01-07,SR3F6,32.000,NaN,NaN,2026-04-14 21:00:00+00:00
2025-01-08,SR3F6,32.000,NaN,NaN,2026-04-14 21:00:00+00:00
...,...,...,...,...,...
2025-11-06,SR3F6,96.340,16.0,1860.0,2026-04-14 21:00:00+00:00
2025-11-07,SR3F6,96.340,2.0,1961.0,2026-04-14 21:00:00+00:00
2025-11-10,SR3F6,96.315,24.0,2031.0,2026-04-14 21:00:00+00:00


Two important things to notice here:
1. The settlement prices seem wrong at the start of the period.
2. During the period we are investigating, there are sensible settlement prices starting in mid July.

The issue with 1 is that our helper function `get_official_stats` does not manage that the `instrument_id` over the time range in question
got reused. At the beginning of the period, it corresponded to a non-`SR3` contract:

In [12]:
old_defs.to_df()

,ts_event,rtype,publisher_id,instrument_id,raw_symbol,security_update_action,instrument_class,min_price_increment,display_factor,expiration,...,sub_fraction,underlying_product,maturity_month,maturity_day,maturity_week,user_defined_instrument,contract_multiplier_unit,flow_schedule_type,tick_rule,symbol
ts_recv,,,,,,,,,,,,,,,,,,,,,
2025-01-02 00:00:00+00:00,2024-12-29 12:04:10.486000+00:00,19,1,42000173,IPTH5,A,F,0.25,0.01,2025-03-20 20:00:00+00:00,...,255,5,3,255,255,N,127,127,255,42000173


Then for 2, we can double-check the `CME` site for the trading calendar for `SR3F6` to find that it did not start trading until July 14th.

https://www.cmegroup.com/markets/interest-rates/stirs/three-month-sofr.calendar.html

In [13]:
instrument_defs.to_df()

,ts_event,rtype,publisher_id,instrument_id,raw_symbol,security_update_action,instrument_class,min_price_increment,display_factor,expiration,...,sub_fraction,underlying_product,maturity_month,maturity_day,maturity_week,user_defined_instrument,contract_multiplier_unit,flow_schedule_type,tick_rule,symbol
ts_recv,,,,,,,,,,,,,,,,,,,,,
2025-11-13 00:00:00+00:00,2025-11-09 12:10:58.349434889+00:00,19,1,42000173,SR3F6,A,F,0.005,0.01,2026-04-14 21:00:00+00:00,...,255,14,1,255,255,N,127,127,255,42000173


This points out how irregularities in the contract months can affect data.

The solution we will use with constant maturity contracts is to only use quarterly `SR3` contracts. Something similar could be done for rolling by
expiration, i.e., define a new continuous method that rolls on expiration but only selecting among quarterly expirations. That is certainly a viable
and useful approach in some situations, but we do not develop it here.

### What are the problems with the following methods for this analysis?

1. Additive price adjustment
2. Multiplicative price adjustment
3. Rolling by volume or open interest

### Constructing Constant Maturity Prices

I restrict to the active quarterly contracts. The monthlies could be used if I
were adjusting settlements, but with OHLC data, there can be issues where a new
quarterly has no trades even though it appears in the instrument table, requiring
more care to filter when a contract becomes active. Also, all the quarterlies
are active at the start of the year, so it is faster to pull the instruments
from a single date.


In [14]:
constant_maturity = tuple(f"{product}.cm.{91 * (i + 1)}" for i in range(num_contracts))
constant_maturity

('SR3.cm.91',
 'SR3.cm.182',
 'SR3.cm.273',
 'SR3.cm.364',
 'SR3.cm.455',
 'SR3.cm.546',
 'SR3.cm.637',
 'SR3.cm.728',
 'SR3.cm.819',
 'SR3.cm.910')

In [15]:
instrument_defs = client.timeseries.get_range(
    dataset="GLBX.MDP3",
    schema="definition",
    symbols=f"{product}.FUT",
    stype_in="parent",
    start=start_of_this_year,
).to_df()
quarters = ["H", "M", "U", "Z"]
quarterlies = instrument_defs[
    instrument_defs["raw_symbol"].str.slice(3, 4).isin(quarters)
]
cm_specs = {
    symbol: get_roll_spec(
        symbol,
        quarterlies.reset_index(),
        start=start_of_this_year,
        end=yesterday,
    )
    for symbol in constant_maturity
}
cm_instruments = {
    int(spec[key])
    for spec_list in cm_specs.values()
    for spec in spec_list
    for key in (
        "p",
        "n",
    )
}
ohlcv = client.timeseries.get_range(
    dataset="GLBX.MDP3",
    schema="ohlcv-1d",
    symbols=cm_instruments,
    stype_in="instrument_id",
    start=start_of_this_year,
    end=yesterday,
).to_df()
expirations = quarterlies[["instrument_id", "expiration"]].drop_duplicates()
ohlcv_with_exp = (
    ohlcv.reset_index()
    .merge(expirations, on="instrument_id", how="left")
    .set_index("ts_event")
)

/var/folders/8y/2f_yzjcd2tz12vswl92n4vpw0000gp/T/ipykernel_18664/3178480485.py:30: BentoWarning:

The streaming request contained one or more days which have reduced quality: 2025-09-17 (degraded), 2025-09-24 (degraded). See: https://databento.com/docs/api-reference-historical/metadata/metadata-get-dataset-condition



In [16]:
cm_pieces = {
    symbol: constant_maturity_splice(
        symbol,
        roll_spec,
        ohlcv_with_exp.reset_index(),
        date_col="ts_event",
        price_col="close",
    )
    for symbol, roll_spec in cm_specs.items()
}

cm_long = [
    pd.wide_to_long(
        df, stubnames=["SR3.cm"], i="ts_event", j="days_to_maturity", sep="."
    )
    for df in cm_pieces.values()
]
cm_close = pd.concat(cm_long)

In [17]:
plot_df = cm_close.reset_index()
fig = px.line(
    plot_df,
    x="ts_event",
    y="SR3.cm",
    color="days_to_maturity",
)
fig.update_layout(title_text=f"Constant maturity contracts, {product}")
fig.show()

An important difference between Eurodollars (the original analysis) and SOFR is the expiration structure. As a 3-month forward rate, the
September contract, for example, reflects the interest rate over the following three months until December. Under the Eurodollar system,
that rate was based on LIBOR quotes, but the LIBOR scandal revealed how that was subject to manipulation. By switching to SOFR, the
September contract rate over the period from September to December gets realized during the life of the futures contract, so
the September contract does not expire until December. However, trading in the contract dies off heavily as expiration approaches,
and we see above that there are breaks in the 91-day constant maturity data. Moreover, the 91-day maturity contract is sensitive to missing
data when 91 days is less than the number of days until the next quarterly maturity.

To avoid these issues, we will start the analysis at 182 days to maturity.

## Patterns in the yield curve

The `SR3` contract is relatively young, starting in 2018 and overtaking Eurodollars (`GE` or `ED`) in trade volume in 2022.
A longer term analysis would want to manage rolling between these two contracts, but for this demonstration we will
go back to 2021

In [18]:
constant_maturity = tuple(f"{product}.cm.{91 * (i + 2)}" for i in range(num_contracts))
constant_maturity

('SR3.cm.182',
 'SR3.cm.273',
 'SR3.cm.364',
 'SR3.cm.455',
 'SR3.cm.546',
 'SR3.cm.637',
 'SR3.cm.728',
 'SR3.cm.819',
 'SR3.cm.910',
 'SR3.cm.1001')

In [19]:
analysis_start = (datetime.date(2021, 1, 1) + us_business_day).date()
analysis_end = (datetime.date(2024, 1, 1) + us_business_day).date()
instrument_defs = client.timeseries.get_range(
    dataset="GLBX.MDP3",
    schema="definition",
    symbols=f"{product}.FUT",
    stype_in="parent",
    start=analysis_start,
    # end=analysis_end,
).to_df()
quarters = ["H", "M", "U", "Z"]
quarterlies = instrument_defs[
    instrument_defs["raw_symbol"].str.slice(3, 4).isin(quarters)
    & (instrument_defs["instrument_class"] == "F")
]

In [20]:
cm_specs = {
    symbol: get_roll_spec(
        symbol,
        quarterlies.reset_index(),
        start=analysis_start,
        end=analysis_end,
    )
    for symbol in constant_maturity
}
cm_instruments = {
    int(spec[key])
    for spec_list in cm_specs.values()
    for spec in spec_list
    for key in (
        "p",
        "n",
    )
}

In [21]:
ohlcv = client.timeseries.get_range(
    dataset="GLBX.MDP3",
    schema="ohlcv-1d",
    symbols=cm_instruments,
    stype_in="instrument_id",
    start=analysis_start,
    end=analysis_end,
).to_df()
expirations = quarterlies[["instrument_id", "expiration"]].drop_duplicates()
ohlcv_with_exp = (
    ohlcv.reset_index()
    .merge(expirations, on="instrument_id", how="left")
    .set_index("ts_event")
)

/var/folders/8y/2f_yzjcd2tz12vswl92n4vpw0000gp/T/ipykernel_18664/2127927488.py:1: BentoWarning:

The streaming request contained one or more days which have reduced quality: 2021-12-05 (degraded), 2022-01-02 (degraded). See: https://databento.com/docs/api-reference-historical/metadata/metadata-get-dataset-condition



In [22]:
cm_pieces = {
    symbol: constant_maturity_splice(
        symbol,
        roll_spec,
        ohlcv_with_exp.reset_index(),
        date_col="ts_event",
        price_col="close",
    )
    for symbol, roll_spec in cm_specs.items()
}

cm_long = [
    pd.wide_to_long(
        df, stubnames=["SR3.cm"], i="ts_event", j="days_to_maturity", sep="."
    )
    for df in cm_pieces.values()
]
cm_close = pd.concat(cm_long)

In [23]:
plot_df = cm_close.reset_index()
fig = px.line(
    plot_df,
    x="ts_event",
    y="SR3.cm",
    color="days_to_maturity",
)
fig.update_layout(title_text=f"Constant maturity contracts, {product}")
fig.show()

The approach to identifying patterns will be independent component analysis.

In [24]:
cm_close_wide = cm_close.reset_index().pivot_table(
    columns="days_to_maturity", values="SR3.cm", index="ts_event"
)
cm_close_wide.columns = [f"dtm_{dtm}" for dtm in cm_close_wide.columns]
cm_close_wide = cm_close_wide.dropna()
cm_close_wide

,dtm_182,dtm_273,dtm_364,dtm_455,dtm_546,dtm_637,dtm_728,dtm_819,dtm_910,dtm_1001
ts_event,,,,,,,,,,
2021-01-04 00:00:00+00:00,99.950000,99.947899,99.940000,99.938949,99.934024,99.929334,99.921402,99.894145,99.864670,99.826670
2021-01-20 00:00:00+00:00,99.945000,99.943071,99.940000,99.938070,99.931416,99.920366,99.903310,99.875663,99.828365,99.758375
2021-02-01 00:00:00+00:00,99.959821,99.947236,99.940000,99.940000,99.937596,99.928389,99.905677,99.875574,99.831339,99.764149
2021-02-02 00:00:00+00:00,99.954712,99.944714,99.937357,99.935000,99.932545,99.923224,99.900320,99.865357,99.815625,99.746121
2021-02-03 00:00:00+00:00,99.954602,99.944604,99.937302,99.935000,99.929987,99.915746,99.887457,99.849949,99.797596,99.725297
...,...,...,...,...,...,...,...,...,...,...
2023-12-26 00:00:00+00:00,95.015962,95.472247,95.874077,96.229856,96.518798,96.717064,96.817341,96.849663,96.843654,96.823991
2023-12-27 00:00:00+00:00,95.057191,95.527870,95.942645,96.293880,96.591051,96.788609,96.893115,96.930000,96.928826,96.914217
2023-12-28 00:00:00+00:00,95.046071,95.501590,95.907437,96.267232,96.563750,96.765263,96.873555,96.910000,96.908661,96.893661


In [25]:
n_components = (2, 3)

signal_dfs = {}
profile_dfs = {}
for n in n_components:
    ica = FastICA(n_components=n, random_state=67)
    signals = ica.fit_transform(cm_close_wide)
    signal_profile = ica.mixing_
    signal_dfs[n] = pd.DataFrame(
        signals, index=cm_close_wide.index, columns=[f"ic_{i}" for i in range(n)]
    )
    profile_dfs[n] = pd.DataFrame(
        signal_profile,
        index=[int(dtm_col.removeprefix("dtm_")) for dtm_col in cm_close_wide.columns],
        columns=[f"ic_{i}" for i in range(n)],
    )

In [26]:
rows = max(n_components)
cols = len(n_components)
fig = make_subplots(
    rows=rows,
    cols=cols,
    shared_xaxes=True,
    vertical_spacing=0.02,
    subplot_titles=[f"IC {n}" for n in range(rows) for j in range(cols)],
)

for j, (_, signal_df) in enumerate(signal_dfs.items(), start=1):
    fig.add_trace(go.Scatter())
    for i, col in enumerate(signal_df.columns, start=1):
        fig.add_trace(
            go.Scatter(x=signal_df.index, y=signal_df[col], name=col, mode="lines"),
            row=i,
            col=j,
        )

    fig.update_layout(
        height=250 * signal_df.shape[1],
        title="Independent Components",
        showlegend=False,
    )

    fig.update_xaxes(title="Time", row=signal_df.shape[1], col=1)
    fig.update_yaxes(title="Value")

fig.show()

In [27]:
days_per_year = 364
fig = make_subplots(
    rows=rows,
    cols=cols,
    shared_xaxes=True,
    vertical_spacing=0.02,
    subplot_titles=[f"IC {n}" for n in range(rows) for j in range(cols)],
)

for j, (_, profile_df) in enumerate(profile_dfs.items(), start=1):
    for i, col in enumerate(profile_df.columns, start=1):
        fig.add_trace(
            go.Scatter(
                x=profile_df.index / days_per_year,
                y=profile_df[col],
                name=col,
                mode="lines",
            ),
            row=i,
            col=j,
        )

    fig.update_layout(
        height=250 * profile_df.shape[1],
        title="Independent Component Weights",
        showlegend=False,
    )

    fig.update_xaxes(title="Years To Maturity", row=profile_df.shape[1], col=1)
    fig.update_yaxes(title="Signal Weights")

fig.show()

## Trading constant maturity signals

As emphasized when we started splicing and adjusting futures prices, the constant maturity prices are not tradable values. As a weighted average,
you might be tempted to approximate the gain or loss from trading a constant maturity (or any other adjusted futures prices) by differencing
the price on entry and exit. Looking at one-day change for example, assuming the previous and next futures contracts
do not change over that day, you can relate the change in constant maturity values to the change in leg prices.
Using the notation from the continuous contract notebook with $s=91$:
$$
91(CM_{m,t+1} - CM_{m,t}) = (91 - (t+1)) F_{p,t+1} + (t+1) F_{n,t+1} - ((91 - t)F_{p,t} - t F_{n,t})
$$
This can be simplified further, but it's an academic exercise. The constant maturity values are NOT tradable prices,
and it's a mistake to use your signal generators as P&L calculators. The demarcation of signal and execution responsibilities
was demonstrated well in the pairs trading notebook and is not pursued further here. The key point is that signals need to
be translated into executions in one or more tradable instruments.